In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI AutoML Image Object Detection


## Installation

Install the latest (preview) version of Vertex SDK.


In [ ]:
! pip3 install -U google-cloud-aiplatform --user

Install the Google *cloud-storage* library as well.


In [ ]:
! pip3 install google-cloud-storage

### Restart the Kernel

Once you've installed the Vertex SDK and Google *cloud-storage*, you need to restart the notebook kernel so it can find the packages.


In [ ]:
import os

if not os.getenv("AUTORUN"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Before you begin

### GPU run-time

*Make sure you're running this notebook in a GPU runtime if you have that option. In Colab, select* **Runtime > Change Runtime Type > GPU**

### Set up your GCP project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a GCP project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project.](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [Enable the Vertex APIs and Compute Engine APIs.](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)

4. [Google Cloud SDK](https://cloud.google.com/sdk) is already installed in Google Cloud Notebooks.

5. Enter your project ID in the cell below. Then run the  cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.


In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for Vertex AI. We recommend when possible, to choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You cannot use a Multi-Regional Storage bucket for training with Vertex. Not all regions provide support for all Vertex services. For the latest support per region, see [Region support for Vertex AI services](https://cloud.google.com/vertex-ai/docs/general/locations)


In [ ]:
REGION = "us-central1"  # @param {type: "string"}

#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append onto the name of resources which will be created in this tutorial.


In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### Authenticate your GCP account

**If you are using Google Cloud Notebooks**, your environment is already
authenticated. Skip this step.

*Note: If you are on an Vertex notebook and run the cell, the cell knows to skip executing the authentication steps.*


In [ ]:
import os
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your Google Cloud account. This provides access
# to your Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on Vertex, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this tutorial in a notebook locally, replace the string
    # below with the path to your service account key and run this cell to
    # authenticate your Google Cloud account.
    else:
        %env GOOGLE_APPLICATION_CREDENTIALS your_path_to_credentials.json

    # Log in to your account on Google Cloud
    ! gcloud auth login

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

This tutorial is designed to use training data that is in a public Cloud Storage bucket and a local Cloud Storage bucket for your batch predictions. You may alternatively use your own training data that you have stored in a local Cloud Storage bucket.

Set the name of your Cloud Storage bucket below. It must be unique across all Cloud Storage buckets.


In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + TIMESTAMP

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.


In [ ]:
! gsutil mb -l $REGION gs://$BUCKET_NAME

Finally, validate access to your Cloud Storage bucket by examining its contents:


In [ ]:
! gsutil ls -al gs://$BUCKET_NAME

### Set up variables

Next, set up some variables used throughout the tutorial.
### Import libraries and define constants


#### Import Vertex SDK

Import the Vertex SDK into our Python environment.


In [ ]:
import os
import sys
import time

from google.cloud.aiplatform import gapic as aip
from google.protobuf import json_format
from google.protobuf.json_format import MessageToJson, ParseDict
from google.protobuf.struct_pb2 import Struct, Value

#### Vertex AI constants

Setup up the following constants for Vertex AI:

- `API_ENDPOINT`: The Vertex AI API service endpoint for dataset, model, job, pipeline and endpoint services.
- `PARENT`: The Vertex AI location root path for dataset, model and endpoint resources.


In [ ]:
# API Endpoint
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)

# Vertex AI location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

#### AutoML constants

Next, setup constants unique to AutoML image classification datasets and training:

- Dataset Schemas: Tells the managed dataset service which type of dataset it is.
- Data Labeling (Annotations) Schemas: Tells the managed dataset service how the data is labeled (annotated).
- Dataset Training Schemas: Tells the Vertex AI Pipelines service the task (e.g., classification) to train the model for.


In [ ]:
# Image Dataset type
IMAGE_SCHEMA = "google-cloud-aiplatform/schema/dataset/metadata/image_1.0.0.yaml"
# Image Labeling type
IMPORT_SCHEMA_IMAGE_OBJECT_DETECTION_BOX = "gs://google-cloud-aiplatform/schema/dataset/ioformat/image_bounding_box_io_format_1.0.0.yaml"
# Image Training task
TRAINING_IMAGE_OBJECT_DETECTION_SCHEMA = "gs://google-cloud-aiplatform/schema/trainingjob/definition/automl_image_object_detection_1.0.0.yaml"

## Clients

The Vertex SDK works as a client/server model. On your side (the Python script) you will create a client that sends requests and receives responses from the server (Vertex).

You will use several clients in this tutorial, so set them all up upfront.

- Dataset Service for managed datasets.
- Model Service for managed models.
- Pipeline Service for training.
- Endpoint Service for deployment.
- Job Service for batch jobs and custom training.
- Prediction Service for serving. *Note*: Prediction has a different service endpoint.


In [ ]:
# client options same for all services
client_options = {"api_endpoint": API_ENDPOINT}


def create_dataset_client():
    client = aip.DatasetServiceClient(client_options=client_options)
    return client


def create_model_client():
    client = aip.ModelServiceClient(client_options=client_options)
    return client


def create_pipeline_client():
    client = aip.PipelineServiceClient(client_options=client_options)
    return client


def create_endpoint_client():
    client = aip.EndpointServiceClient(client_options=client_options)
    return client


def create_prediction_client():
    client = aip.PredictionServiceClient(client_options=client_options)
    return client


def create_job_client():
    client = aip.JobServiceClient(client_options=client_options)
    return client


clients = {}
clients["dataset"] = create_dataset_client()
clients["model"] = create_model_client()
clients["pipeline"] = create_pipeline_client()
clients["endpoint"] = create_endpoint_client()
clients["prediction"] = create_prediction_client()
clients["job"] = create_job_client()

for client in clients.items():
    print(client)

In [ ]:
IMPORT_FILE = "gs://cloud-ml-data/img/openimage/csv/salads_ml_use.csv"

In [ ]:
! gsutil cat $IMPORT_FILE | head -n 10

*Example output*:
```
TEST,gs://cloud-ml-data/img/openimage/103/279324025_3e74a32a84_o.jpg,Baked Goods,0.005743,0.084985,,,0.567511,0.735736,,
TEST,gs://cloud-ml-data/img/openimage/103/279324025_3e74a32a84_o.jpg,Salad,0.402759,0.310473,,,1.000000,0.982695,,
TEST,gs://cloud-ml-data/img/openimage/1064/3167707458_7b2eebed9e_o.jpg,Cheese,0.000000,0.000000,,,0.054865,0.480665,,
TEST,gs://cloud-ml-data/img/openimage/1064/3167707458_7b2eebed9e_o.jpg,Cheese,0.041131,0.401678,,,0.318230,0.785916,,
TEST,gs://cloud-ml-data/img/openimage/1064/3167707458_7b2eebed9e_o.jpg,Cheese,0.116263,0.065161,,,0.451528,0.286489,,
TEST,gs://cloud-ml-data/img/openimage/1064/3167707458_7b2eebed9e_o.jpg,Cheese,0.557359,0.411551,,,0.988760,0.731613,,
TEST,gs://cloud-ml-data/img/openimage/1064/3167707458_7b2eebed9e_o.jpg,Cheese,0.562206,0.059401,,,0.876467,0.260982,,
TEST,gs://cloud-ml-data/img/openimage/1064/3167707458_7b2eebed9e_o.jpg,Cheese,0.567861,0.000161,,,0.699543,0.077502,,
TEST,gs://cloud-ml-data/img/openimage/1064/3167707458_7b2eebed9e_o.jpg,Cheese,0.916052,0.085569,,,1.000000,0.348036,,
TEST,gs://cloud-ml-data/img/openimage/1064/3167707458_7b2eebed9e_o.jpg,Salad,0.000000,0.000000,,,1.000000,1.000000,,
```


## Create a dataset


### [projects.locations.datasets.create](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.datasets/create)


#### Request


In [ ]:
DATA_SCHEMA = IMAGE_SCHEMA

dataset = {
    "display_name": "salads_" + TIMESTAMP,
    "metadata_schema_uri": "gs://" + DATA_SCHEMA,
}

print(
    MessageToJson(
        aip.CreateDatasetRequest(
            parent=PARENT,
            dataset=dataset,
        ).__dict__["_pb"]
    )
)

*Example output*:
```
{
  "parent": "projects/migration-ucaip-training/locations/us-central1",
  "dataset": {
    "displayName": "salads_20210226015226",
    "metadataSchemaUri": "gs://google-cloud-aiplatform/schema/dataset/metadata/image_1.0.0.yaml"
  }
}
```


#### Call


In [ ]:
request = clients["dataset"].create_dataset(
    parent=PARENT,
    dataset=dataset,
)

#### Response


In [ ]:
result = request.result()

print(MessageToJson(result.__dict__["_pb"]))

*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/datasets/8577474926234042368",
  "displayName": "salads_20210226015226",
  "metadataSchemaUri": "gs://google-cloud-aiplatform/schema/dataset/metadata/image_1.0.0.yaml",
  "labels": {
    "aiplatform.googleapis.com/dataset_metadata_schema": "IMAGE"
  },
  "metadata": {
    "dataItemSchemaUri": "gs://google-cloud-aiplatform/schema/dataset/dataitem/image_1.0.0.yaml"
  }
}
```


In [ ]:
# The full unique ID for the dataset
dataset_id = result.name
# The short numeric ID for the dataset
dataset_short_id = dataset_id.split("/")[-1]

print(dataset_id)

### [projects.locations.datasets.import](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.datasets/import)


#### Request


In [ ]:
LABEL_SCHEMA = IMPORT_SCHEMA_IMAGE_OBJECT_DETECTION_BOX

import_config = {
    "gcs_source": {
        "uris": [IMPORT_FILE],
    },
    "import_schema_uri": LABEL_SCHEMA,
}

print(
    MessageToJson(
        aip.ImportDataRequest(
            name=dataset_short_id,
            import_configs=[import_config],
        ).__dict__["_pb"]
    )
)

*Example output*:
```
{
  "name": "8577474926234042368",
  "importConfigs": [
    {
      "gcsSource": {
        "uris": [
          "gs://cloud-ml-data/img/openimage/csv/salads_ml_use.csv"
        ]
      },
      "importSchemaUri": "gs://google-cloud-aiplatform/schema/dataset/ioformat/image_bounding_box_io_format_1.0.0.yaml"
    }
  ]
}
```


#### Call


In [ ]:
request = clients["dataset"].import_data(
    name=dataset_id,
    import_configs=[import_config],
)

#### Response


In [ ]:
result = request.result()

print(MessageToJson(result.__dict__["_pb"]))

*Example output*:
```
{}
```


## Train a model


### [projects.locations.trainingPipelines.create](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.trainingPipelines/create)


#### Request


In [ ]:
TRAINING_SCHEMA = TRAINING_IMAGE_OBJECT_DETECTION_SCHEMA

task = Value(
    struct_value=Struct(
        fields={
            "budget_milli_node_hours": Value(number_value=20000),
            "disable_early_stopping": Value(bool_value=False),
        },
    )
)

training_pipeline = {
    "display_name": "salads_" + TIMESTAMP,
    "input_data_config": {
        "dataset_id": dataset_short_id,
    },
    "model_to_upload": {
        "display_name": "salads_" + TIMESTAMP,
    },
    "training_task_definition": TRAINING_SCHEMA,
    "training_task_inputs": task,
}

print(
    MessageToJson(
        aip.CreateTrainingPipelineRequest(
            parent=PARENT,
            training_pipeline=training_pipeline,
        ).__dict__["_pb"]
    )
)

*Example output*:
```
{
  "parent": "projects/migration-ucaip-training/locations/us-central1",
  "trainingPipeline": {
    "displayName": "salads_20210226015226",
    "inputDataConfig": {
      "datasetId": "8577474926234042368"
    },
    "trainingTaskDefinition": "gs://google-cloud-aiplatform/schema/trainingjob/definition/automl_image_object_detection_1.0.0.yaml",
    "trainingTaskInputs": {
      "budget_milli_node_hours": 20000.0,
      "disable_early_stopping": false
    },
    "modelToUpload": {
      "displayName": "salads_20210226015226"
    }
  }
}
```


#### Call


In [ ]:
request = clients["pipeline"].create_training_pipeline(
    parent=PARENT,
    training_pipeline=training_pipeline,
)

#### Response


In [ ]:
print(MessageToJson(request.__dict__["_pb"]))

*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/trainingPipelines/2049683188220952576",
  "displayName": "salads_20210226015226",
  "inputDataConfig": {
    "datasetId": "8577474926234042368"
  },
  "trainingTaskDefinition": "gs://google-cloud-aiplatform/schema/trainingjob/definition/automl_image_object_detection_1.0.0.yaml",
  "trainingTaskInputs": {
    "budgetMilliNodeHours": "20000"
  },
  "modelToUpload": {
    "displayName": "salads_20210226015226"
  },
  "state": "PIPELINE_STATE_PENDING",
  "createTime": "2021-02-26T02:12:41.612146Z",
  "updateTime": "2021-02-26T02:12:41.612146Z"
}
```


In [ ]:
# The full unique ID for the training pipeline
training_pipeline_id = request.name
# The short numeric ID for the training pipeline
training_pipeline_short_id = training_pipeline_id.split("/")[-1]

print(training_pipeline_id)

### [projects.locations.trainingPipelines.get](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.trainingPipelines/get)


#### Call


In [ ]:
request = clients["pipeline"].get_training_pipeline(
    name=training_pipeline_id,
)

#### Response


In [ ]:
print(MessageToJson(request.__dict__["_pb"]))

*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/trainingPipelines/2049683188220952576",
  "displayName": "salads_20210226015226",
  "inputDataConfig": {
    "datasetId": "8577474926234042368"
  },
  "trainingTaskDefinition": "gs://google-cloud-aiplatform/schema/trainingjob/definition/automl_image_object_detection_1.0.0.yaml",
  "trainingTaskInputs": {
    "budgetMilliNodeHours": "20000"
  },
  "modelToUpload": {
    "displayName": "salads_20210226015226"
  },
  "state": "PIPELINE_STATE_PENDING",
  "createTime": "2021-02-26T02:12:41.612146Z",
  "updateTime": "2021-02-26T02:12:41.612146Z"
}
```


In [ ]:
while True:
    response = clients["pipeline"].get_training_pipeline(name=training_pipeline_id)
    if response.state != aip.PipelineState.PIPELINE_STATE_SUCCEEDED:
        print("Training job has not completed:", response.state)
        model_to_deploy_name = None
        if response.state == aip.PipelineState.PIPELINE_STATE_FAILED:
            break
    else:
        model_id = response.model_to_upload.name
        print("Training Time:", response.end_time - response.start_time)
        break
    time.sleep(20)

print(model_id)

## Evaluate the model


### [projects.locations.models.evaluations.list](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.models.evaluations/list)


#### Call


In [ ]:
request = clients["model"].list_model_evaluations(parent=model_id)

#### Response


In [ ]:
import json

model_evaluations = [json.loads(MessageToJson(me.__dict__["_pb"])) for me in request]
# The evaluation slice
evaluation_slice = request.model_evaluations[0].name

print(json.dumps(model_evaluations, indent=2))

*Example output*:
```

[
  {
    "name": "projects/116273516712/locations/us-central1/models/770273865954754560/evaluations/7557961565471768576",
    "metricsSchemaUri": "gs://google-cloud-aiplatform/schema/modelevaluation/image_object_detection_metrics_1.0.0.yaml",
    "metrics": {
      "boundingBoxMetrics": [
        {
          "meanAveragePrecision": 0.37167007,
          "confidenceMetrics": [
            {
              "precision": 0.09565217,
              "f1Score": 0.15985467,
              "confidenceThreshold": 4.8275826e-05,
              "recall": 0.48618785
            },
            {
              "confidenceThreshold": 0.0007978445,
              "f1Score": 0.18373811,
              "precision": 0.11357702,
              "recall": 0.48066297
            },
            
            # REMOVED FOR BREVITY
            
            {
              "f1Score": 0.043243244,
              "precision": 1.0,
              "confidenceThreshold": 0.9953122,
              "recall": 0.022099448
            },
            {
              "confidenceThreshold": 0.99533135,
              "precision": 1.0,
              "recall": 0.016574586,
              "f1Score": 0.032608695
            },
            {
              "f1Score": 0.021857925,
              "confidenceThreshold": 0.99550796,
              "precision": 1.0,
              "recall": 0.011049724
            },
            {
              "confidenceThreshold": 0.99619305,
              "f1Score": 0.010989011,
              "precision": 1.0,
              "recall": 0.005524862
            }
          ],
          "iouThreshold": 0.45,
          "meanAveragePrecision": 0.32951096
        }
      ],
      "evaluatedBoundingBoxCount": 181.0,
      "boundingBoxMeanAveragePrecision": 0.2927905
    },
    "createTime": "2021-02-26T03:38:42.086497Z",
    "sliceDimensions": [
      "annotationSpec"
    ]
  }
]
```


### [projects.locations.models.evaluations.get](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.models.evaluations/get)


#### Call


In [ ]:
request = clients["model"].get_model_evaluation(
    name=evaluation_slice,
)

#### Response


In [ ]:
print(MessageToJson(request.__dict__["_pb"]))

*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/models/770273865954754560/evaluations/7557961565471768576",
  "metricsSchemaUri": "gs://google-cloud-aiplatform/schema/modelevaluation/image_object_detection_metrics_1.0.0.yaml",
  "metrics": {
    "boundingBoxMeanAveragePrecision": 0.2927905,
    "evaluatedBoundingBoxCount": 181.0,
    "boundingBoxMetrics": [
      {
        "meanAveragePrecision": 0.37167007,
        "confidenceMetrics": [
          {
            "f1Score": 0.15985467,
            "confidenceThreshold": 4.8275826e-05,
            "precision": 0.09565217,
            "recall": 0.48618785
          },
          {
            "precision": 0.11357702,
            "confidenceThreshold": 0.0007978445,
            "f1Score": 0.18373811,
            "recall": 0.48066297
          },
          {
            "confidenceThreshold": 0.003912397,
            "precision": 0.15167548,
            "recall": 0.47513813,
            "f1Score": 0.22994651
          },
          
          # REMOVED FOR BREVITY
          
          {
            "precision": 1.0,
            "confidenceThreshold": 0.9953122,
            "f1Score": 0.043243244,
            "recall": 0.022099448
          },
          {
            "precision": 1.0,
            "f1Score": 0.032608695,
            "recall": 0.016574586,
            "confidenceThreshold": 0.99533135
          },
          {
            "recall": 0.011049724,
            "precision": 1.0,
            "confidenceThreshold": 0.99550796,
            "f1Score": 0.021857925
          },
          {
            "f1Score": 0.010989011,
            "precision": 1.0,
            "recall": 0.005524862,
            "confidenceThreshold": 0.99619305
          }
        ],
        "meanAveragePrecision": 0.32951096,
        "iouThreshold": 0.45
      }
    ]
  },
  "createTime": "2021-02-26T03:38:42.086497Z",
  "sliceDimensions": [
    "annotationSpec"
  ]
}
```


## Make batch predictions


### Make the batch input file

Let's now make a batch input file, which you store in your local Cloud Storage bucket. The batch input file can be either CSV or JSONL. You will use JSONL in this tutorial. For JSONL file, you make one dictionary entry per line for each image. The dictionary contains the key/value pairs:

- `content`: The Cloud Storage path to the image.
- `mimeType`: The content type. In our example, it is an `image/jpeg` file.


In [ ]:
test_items = ! gsutil cat $IMPORT_FILE | head -n2

test_item_1, test_label_1 = test_items[0].split(",")[1], test_items[0].split(",")[2]
test_item_2, test_label_2 = test_items[0].split(",")[1], test_items[0].split(",")[2]

file_1 = test_item_1.split("/")[-1]
file_2 = test_item_2.split("/")[-1]

! gsutil cp $test_item_1 gs://$BUCKET_NAME/$file_1
! gsutil cp $test_item_2 gs://$BUCKET_NAME/$file_2

test_item_1 = "gs://" + BUCKET_NAME + "/" + file_1
test_item_2 = "gs://" + BUCKET_NAME + "/" + file_2

print(test_item_1, test_label_1)
print(test_item_2, test_label_2)

*Example output*:
```
gs://migration-ucaip-trainingaip-20210226015226/279324025_3e74a32a84_o.jpg Baked Goods
gs://migration-ucaip-trainingaip-20210226015226/279324025_3e74a32a84_o.jpg Baked Goods
```


In [ ]:
import json

import tensorflow as tf

gcs_input_uri = "gs://" + BUCKET_NAME + "/test.jsonl"
with tf.io.gfile.GFile(gcs_input_uri, "w") as f:
    data = {"content": test_item_1, "mime_type": "image/jpeg"}
    f.write(json.dumps(data) + "\n")
    data = {"content": test_item_2, "mime_type": "image/jpeg"}
    f.write(json.dumps(data) + "\n")

!gsutil cat $gcs_input_uri

*Example output*:
```
{"content": "gs://migration-ucaip-trainingaip-20210226015226/279324025_3e74a32a84_o.jpg", "mime_type": "image/jpeg"}
{"content": "gs://migration-ucaip-trainingaip-20210226015226/279324025_3e74a32a84_o.jpg", "mime_type": "image/jpeg"}
```


### [projects.locations.batchPredictionJobs.create](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.batchPredictionJobs/create)


#### Request


In [ ]:
parameters = {"confidenceThreshold": 0.5, "maxPredictions": 2}

batch_prediction_job = {
    "display_name": "salads_" + TIMESTAMP,
    "model": model_id,
    "input_config": {
        "instances_format": "jsonl",
        "gcs_source": {
            "uris": [gcs_input_uri],
        },
    },
    "model_parameters": json_format.ParseDict(parameters, Value()),
    "output_config": {
        "predictions_format": "jsonl",
        "gcs_destination": {
            "output_uri_prefix": "gs://" + f"{BUCKET_NAME}/batch_output/",
        },
    },
    "dedicated_resources": {
        "machine_spec": {
            "machine_type": "n1-standard-2",
            "accelerator_type": 0,
        },
        "starting_replica_count": 1,
        "max_replica_count": 1,
    },
}

print(
    MessageToJson(
        aip.CreateBatchPredictionJobRequest(
            parent=PARENT, batch_prediction_job=batch_prediction_job
        ).__dict__["_pb"]
    )
)

*Example output*:
```
{
  "parent": "projects/migration-ucaip-training/locations/us-central1",
  "batchPredictionJob": {
    "displayName": "salads_20210226015226",
    "model": "projects/116273516712/locations/us-central1/models/770273865954754560",
    "inputConfig": {
      "instancesFormat": "jsonl",
      "gcsSource": {
        "uris": [
          "gs://migration-ucaip-trainingaip-20210226015226/test.jsonl"
        ]
      }
    },
    "modelParameters": {
      "confidenceThreshold": 0.5,
      "maxPredictions": 2.0
    },
    "outputConfig": {
      "predictionsFormat": "jsonl",
      "gcsDestination": {
        "outputUriPrefix": "gs://migration-ucaip-trainingaip-20210226015226/batch_output/"
      }
    },
    "dedicatedResources": {
      "machineSpec": {
        "machineType": "n1-standard-2"
      },
      "startingReplicaCount": 1,
      "maxReplicaCount": 1
    }
  }
}
```


#### Call


In [ ]:
request = clients["job"].create_batch_prediction_job(
    parent=PARENT,
    batch_prediction_job=batch_prediction_job,
)

#### Response


In [ ]:
print(MessageToJson(request.__dict__["_pb"]))

*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/batchPredictionJobs/2404341658876379136",
  "displayName": "salads_20210226015226",
  "model": "projects/116273516712/locations/us-central1/models/770273865954754560",
  "inputConfig": {
    "instancesFormat": "jsonl",
    "gcsSource": {
      "uris": [
        "gs://migration-ucaip-trainingaip-20210226015226/test.jsonl"
      ]
    }
  },
  "modelParameters": {
    "maxPredictions": 2.0,
    "confidenceThreshold": 0.5
  },
  "outputConfig": {
    "predictionsFormat": "jsonl",
    "gcsDestination": {
      "outputUriPrefix": "gs://migration-ucaip-trainingaip-20210226015226/batch_output/"
    }
  },
  "state": "JOB_STATE_PENDING",
  "completionStats": {
    "incompleteCount": "-1"
  },
  "createTime": "2021-02-26T09:36:17.046416Z",
  "updateTime": "2021-02-26T09:36:17.046416Z"
}
```


In [ ]:
# The fully qualified ID for the batch job
batch_job_id = request.name
# The short numeric ID for the batch job
batch_job_short_id = batch_job_id.split("/")[-1]

print(batch_job_id)

### [projects.locations.batchPredictionJobs.get](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.batchPredictionJobs/get)


#### Call


In [ ]:
request = clients["job"].get_batch_prediction_job(
    name=batch_job_id,
)

#### Response


In [ ]:
print(MessageToJson(request.__dict__["_pb"]))

*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/batchPredictionJobs/2404341658876379136",
  "displayName": "salads_20210226015226",
  "model": "projects/116273516712/locations/us-central1/models/770273865954754560",
  "inputConfig": {
    "instancesFormat": "jsonl",
    "gcsSource": {
      "uris": [
        "gs://migration-ucaip-trainingaip-20210226015226/test.jsonl"
      ]
    }
  },
  "modelParameters": {
    "maxPredictions": 2.0,
    "confidenceThreshold": 0.5
  },
  "outputConfig": {
    "predictionsFormat": "jsonl",
    "gcsDestination": {
      "outputUriPrefix": "gs://migration-ucaip-trainingaip-20210226015226/batch_output/"
    }
  },
  "state": "JOB_STATE_PENDING",
  "completionStats": {
    "incompleteCount": "-1"
  },
  "createTime": "2021-02-26T09:36:17.046416Z",
  "updateTime": "2021-02-26T09:36:17.046416Z"
}
```


In [ ]:
def get_latest_predictions(gcs_out_dir):
    """ Get the latest prediction subfolder using the timestamp in the subfolder name"""
    folders = !gsutil ls $gcs_out_dir
    latest = ""
    for folder in folders:
        subfolder = folder.split("/")[-2]
        if subfolder.startswith("prediction-"):
            if subfolder > latest:
                latest = folder[:-1]
    return latest


while True:
    response = clients["job"].get_batch_prediction_job(name=batch_job_id)
    if response.state != aip.JobState.JOB_STATE_SUCCEEDED:
        print("The job has not completed:", response.state)
        if response.state == aip.JobState.JOB_STATE_FAILED:
            break
    else:
        folder = get_latest_predictions(
            response.output_config.gcs_destination.output_uri_prefix
        )
        ! gsutil ls $folder/prediction*.jsonl

        ! gsutil cat $folder/prediction*.jsonl
        break
    time.sleep(60)

*Example output*:
```
gs://migration-ucaip-trainingaip-20210226015226/batch_output/prediction-salads_20210226015226-2021-02-26T09:36:16.878261Z/predictions_00001.jsonl
{"instance":{"content":"gs://migration-ucaip-trainingaip-20210226015226/279324025_3e74a32a84_o.jpg","mimeType":"image/jpeg"},"prediction":{"ids":["7754337640727445504","8330798393030868992"],"displayNames":["Salad","Baked Goods"],"confidences":[0.99217236,0.93992615],"bboxes":[[0.382205,0.9760891,0.29858154,0.9979937],[0.0012550354,0.5893767,0.06807296,0.81340706]]}}
{"instance":{"content":"gs://migration-ucaip-trainingaip-20210226015226/279324025_3e74a32a84_o.jpg","mimeType":"image/jpeg"},"prediction":{"ids":["7754337640727445504","8330798393030868992"],"displayNames":["Salad","Baked Goods"],"confidences":[0.99217236,0.93992615],"bboxes":[[0.382205,0.9760891,0.29858154,0.9979937],[0.0012550354,0.5893767,0.06807296,0.81340706]]}}
```


## Make online predictions

### Prepare file for online prediction

### [projects.locations.endpoints.create](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.endpoints/create)

#### Request

In [ ]:
endpoint = {
    "display_name": "salads_" + TIMESTAMP,
}

print(
    MessageToJson(
        aip.CreateEndpointRequest(
            parent=PARENT,
            endpoint=endpoint,
        ).__dict__["_pb"]
    )
)

*Example output*:
```
{
  "parent": "projects/migration-ucaip-training/locations/us-central1",
  "endpoint": {
    "displayName": "salads_20210226015226"
  }
}
```


#### Call

In [ ]:
request = clients["endpoint"].create_endpoint(
    parent=PARENT,
    endpoint=endpoint,
)

#### Response

In [ ]:
result = request.result()

print(MessageToJson(result.__dict__["_pb"]))

*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/endpoints/2449782275429105664"
}
```


In [ ]:
# The full unique ID for the endpoint
endpoint_id = result.name
# The short numeric ID for the endpoint
endpoint_short_id = endpoint_id.split("/")[-1]

print(endpoint_id)

### [projects.locations.endpoints.deployModel](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.endpoints/deployModel)

#### Request

In [ ]:
deployed_model = {
    "model": model_id,
    "display_name": "salads_" + TIMESTAMP,
    "automatic_resources": {
        "min_replica_count": 1,
        "max_replica_count": 1,
    },
}

traffic_split = {
    "0": 100,
}

print(
    MessageToJson(
        aip.DeployModelRequest(
            endpoint=endpoint_id,
            deployed_model=deployed_model,
            traffic_split=traffic_split,
        ).__dict__["_pb"]
    )
)

*Example output*:
```
{
  "endpoint": "projects/116273516712/locations/us-central1/endpoints/2449782275429105664",
  "deployedModel": {
    "model": "projects/116273516712/locations/us-central1/models/770273865954754560",
    "displayName": "salads_20210226015226",
    "automaticResources": {
      "minReplicaCount": 1,
      "maxReplicaCount": 1
    }
  },
  "trafficSplit": {
    "0": 100
  }
}
```


#### Call

In [ ]:
request = clients["endpoint"].deploy_model(
    endpoint=endpoint_id,
    deployed_model=deployed_model,
    traffic_split=traffic_split,
)

#### Response

In [ ]:
result = request.result()

print(MessageToJson(result.__dict__["_pb"]))

*Example output*:
```
{
  "deployedModel": {
    "id": "3904304217581420544"
  }
}
```


In [ ]:
# The unique ID for the deployed model
deployed_model_id = result.deployed_model.id

print(deployed_model_id)

### [projects.locations.endpoints.predict](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.endpoints/predict)


#### Request

In [ ]:
import base64

import tensorflow as tf

single_file = ! gsutil cat $IMPORT_FILE | head -n 1
single_file = single_file[0].split(",")[1]

with tf.io.gfile.GFile(single_file, "rb") as f:
    content = f.read()

instances_list = [{"content": base64.b64encode(content).decode("utf-8")}]
instances = [json_format.ParseDict(s, Value()) for s in instances_list]

parameters_dict = {
    "confidenceThreshold": 0.5,
    "maxPredictions": 2,
}
parameters = json_format.ParseDict(parameters_dict, Value())

request = aip.PredictRequest(
    endpoint=endpoint_id,
    parameters=parameters,
)
request.instances.append(instances)

print(MessageToJson(request.__dict__["_pb"]))

*Example output*:
```
{
  "endpoint": "projects/116273516712/locations/us-central1/endpoints/2449782275429105664",
  "instances": [
    [
      {
        "content": "/9j/4RtSRXhpZgAASUkqAAgAAAAIAA8BAgAGAAAAbgAAABABAgAEAAAATjkxABIBAwABAAAAAQAAABoBBQABAAAAdAAAABsBBQABAAAAfAAAACgBAwABAAAAAgAAABMCAwABAAAAAQAAAGmHBAABAAAAVh+71qaC5bdjJzRRdnuZVI80S+khxk0/zzXWpHmyjqPW4IHWnC755rWM3HW5DiH2jJzmke6296n22o+W4xtQIPWmrqJJPNVCr/X9MXJ1JPtW4Uvm+9N7iI3n9TTBcc9axqTa0EyObUUiXrVKTV0ZvvVjb
# REMOVED for brevity
KUtSNYfMbpVhYdpFZxXcL66F23TjcRnFTTzgLwabmuYbXUqlgDmoHmy2BVpqw29UVLqbqKxrqcKDVWtqFrGTe6l8pVelYF5KZmq7JvQ00iiFU5qxGTkfWtouzsS2f/Z"
      }
    ]
  ],
  "parameters": {
    "confidenceThreshold": 0.5,
    "maxPredictions": 2.0
  }
}
```


#### Call


In [ ]:
request = clients["prediction"].predict(
    endpoint=endpoint_id,
    instances=instances,
    parameters=parameters,
)

#### Response


In [ ]:
print(MessageToJson(request.__dict__["_pb"]))

*Example output*:
```
{
  "predictions": [
    {
      "ids": [
        "7754337640727445504",
        "8330798393030868992"
      ],
      "confidences": [
        0.99217236,
        0.939926147
      ],
      "displayNames": [
        "Salad",
        "Baked Goods"
      ],
      "bboxes": [
        [
          0.38220492,
          0.976089239,
          0.298581541,
          0.997993708
        ],
        [
          0.00125509501,
          0.589376688,
          0.0680729151,
          0.813407123
        ]
      ]
    }
  ],
  "deployedModelId": "3904304217581420544"
}
```


### [projects.locations.endpoints.undeployModel](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.endpoints/undeployModel)


#### Call


In [ ]:
request = clients["endpoint"].undeploy_model(
    endpoint=endpoint_id,
    deployed_model_id=deployed_model_id,
    traffic_split={},
)

#### Response


In [ ]:
result = request.result()

print(MessageToJson(result.__dict__["_pb"]))

*Example output*:
```
{}
```


# Cleaning up

To clean up all GCP resources used in this project, you can [delete the GCP
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial.


In [ ]:
delete_dataset = True
delete_model = True
delete_endpoint = True
delete_pipeline = True
delete_batchjob = True
delete_bucket = True

# Delete the dataset using the Vertex AI fully qualified identifier for the dataset
try:
    if delete_dataset:
        clients["dataset"].delete_dataset(name=dataset_id)
except Exception as e:
    print(e)

# Delete the model using the Vertex AI fully qualified identifier for the model
try:
    if delete_model:
        clients["model"].delete_model(name=model_id)
except Exception as e:
    print(e)

# Delete the endpoint using the Vertex AI fully qualified identifier for the endpoint
try:
    if delete_endpoint:
        clients["endpoint"].delete_endpoint(name=endpoint_id)
except Exception as e:
    print(e)

# Delete the training pipeline using the Vertex AI fully qualified identifier for the training pipeline
try:
    if delete_pipeline:
        clients["pipeline"].delete_training_pipeline(name=training_pipeline_id)
except Exception as e:
    print(e)

# Delete the batch job using the Vertex AI fully qualified identifier for the batch job
try:
    if delete_batchjob:
        clients["job"].delete_batch_prediction_job(name=batch_job_id)
except Exception as e:
    print(e)

if delete_bucket and "BUCKET_NAME" in globals():
    ! gsutil rm -r gs://$BUCKET_NAME